## PREDICI

In [ ]:
from Classe_sismogramma_v3 import ClasseDataset
from tensorflow import keras
from keras.layers import  Dropout
import numpy as np
import pandas as pd
import gc

class MCDropout(Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

def predici(modello, dati, pred, i, batch=2048):
    y_predicted = modello.predict(dati, batch_size=batch)
    y_predicted = np.reshape(y_predicted,len(y_predicted))
    pred[f"pred_n_{i}"] = y_predicted

In [ ]:
hdf5_predicting = "/home/silvia/Desktop/Instance_Data/Undecidable/Instance_undecidable_data_tot_no0_semiamp_80.hdf5"
csv_predicting = "/home/silvia/Desktop/Instance_Data/Undecidable/Instance_undecidable_metadata_tot_no0.csv"

Data_predicting = ClasseDataset()
Data_predicting.leggi_custom_dataset(hdf5_predicting, csv_predicting)
sample_train = len(Data_predicting.sismogramma)
print("Ho Letto dataset")
lung = len(Data_predicting.sismogramma[0])
semi_amp = 80

In [ ]:
impostazione = "8"          # quale rete prendere (per ora More 1 o More 2 o etc)
replica = "4"               # scelgo rete More_impostazione / replica
quante_predizioni = 7
salva_predizioni = True
time_shift = 0

tent = f"More_{impostazione}_{replica}"
model_pat = f'/home/silvia/Documents/GitHub/primoprogetto/Codici/Tentativi/More_{impostazione}/{replica}/Tentativo_{tent}.hdf5'
nome_predizione = f"/home/silvia/Documents/GitHub/primoprogetto/Codici/Tentativi/Bag_predictions/MCDrop_Instance_undecidable_{tent}_vers_3.csv"

xtest = Data_predicting.sismogramma

print("XSHAPEEEEEEEEEEEE", xtest.shape)
model = keras.models.load_model(model_pat)
model.summary()

model_MCD = keras.models.Sequential([
        model.get_layer(index=0),
        MCDropout(0.5),
        model.get_layer(index=2),
        model.get_layer(index=3),
        model.get_layer(index=4),
        model.get_layer(index=5),
        model.get_layer(index=6),
        MCDropout(0.5),
        model.get_layer(index=8),
        model.get_layer(index=9),
        model.get_layer(index=10),
        model.get_layer(index=11),
        model.get_layer(index=12),
    ])
model_MCD.summary()
pred = pd.DataFrame.from_dict({"traccia":Data_predicting.metadata["trace_name"], "y_mano": "unndecidable"})

for i in range(quante_predizioni):
    predici(model_MCD, xtest, pred, i, batch=2048)
    for k in range(15):
        gc.collect()
    """y_predicted = model_MCD.predict(xtest, batch_size=2048)
    y_predicted = np.reshape(y_predicted,len(y_predicted))
    pred[f"pred_n_{i}"] = y_predicted"""


if salva_predizioni:
    pred.to_csv(nome_predizione, index=False)

## Visualizza predizioni

In [ ]:
# MCDropout Mean prediction
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import PercentFormatter

nbins = 40
pat_und = "/home/silvia/Documents/GitHub/primoprogetto/Codici/Tentativi/Bag_predictions/MCDrop_Instance_undecidable_More_8_4_vers_3.csv"
datapd_und = pd.read_csv(pat_und)
datapd_und = datapd_und.drop(["traccia", "y_mano"], axis=1)
mean_und = datapd_und.mean(axis=1)

fig, axi = plt.subplots(1, 2, figsize=(13, 5))

mean_und.hist(bins=nbins, edgecolor="black", ax=axi[1], weights=np.ones(len(mean_und)) / len(mean_und))
axi[1].yaxis.set_major_formatter(PercentFormatter(1))
axi[1].set_title("Waveforms labeled as undecidable", wrap=True)
plt.show()

In [ ]:
import pandas as pd 


pat_und1 = "/home/silvia/Documents/GitHub/primoprogetto/Codici/Tentativi/Bag_predictions/MCDrop_Instance_undecidable_More_8_4.csv"
datapd_und1 = pd.read_csv(pat_und1)

pat_und2 = "/home/silvia/Documents/GitHub/primoprogetto/Codici/Tentativi/Bag_predictions/MCDrop_Instance_undecidable_More_8_4_vers_2.csv"
datapd_und2 = pd.read_csv(pat_und2)

In [ ]:
datapd_und1